In [6]:
import sys
sys.path.insert(0, '../..')
import copy
import numpy as np
from model.base import CovidModel, SimulationParameters, set_parameters, normal_ci
from utils import BasicStatistics, RemovePolicy, Propaganda, setup_city_layout, AddPolicyInfectedRate, confidence_interval, multiple_runs
from model.utils import SocialPolicy

In [7]:
listeners =[] 
strings = """ [
["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_ELEMENTARY_SCHOOL,0.00025 ,11],
["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_MIDDLE_SCHOOL, 0.00025 ,11],
["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_HIGH_SCHOOL, 0.00025 ,11],
["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_RETAIL,0.00025 ,11],
["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_FACTORY, 0.00025 ,11],
["AddPolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_OFFICE, 0.00025 ,11],
["RemovePolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_ELEMENTARY_SCHOOL, 0.00025 ,21],
["RemovePolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_MIDDLE_SCHOOL, 0.00025 ,21],
["RemovePolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_HIGH_SCHOOL,0.00025 ,21],
["RemovePolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_RETAIL, 0.00025 ,21],
["RemovePolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_FACTORY, 0.00025 ,21],
["RemovePolicyInfectedRateWindow", SocialPolicy.LOCKDOWN_OFFICE,0.00025 ,21]
]"""

In [8]:
common_parameters = SimulationParameters(
    mask_user_rate = 0.9, # 0.0,
    mask_efficacy = 0.6, #0.0,
    imune_rate = 0.01,
    initial_infection_rate = 0.01,
    hospitalization_capacity = 0.05,
    latency_period_shape = 3,
    latency_period_scale = 1, 
    incubation_period_shape = 6,
    incubation_period_scale = 1, 
    mild_period_duration_shape = 14,
    mild_period_duration_scale = 1,
    hospitalization_period_duration_shape = 12,
    hospitalization_period_duration_scale = 1,
    symptomatic_isolation_rate = 0.0,
    asymptomatic_contagion_probability = 0.1,
#    asymptomatic_contagion_probability = 0.05,
    risk_tolerance_mean = 0.2,
    risk_tolerance_stdev = 0.01,
    #herding_behavior_mean = 0.7,
    herding_behavior_mean = 0.1,
    herding_behavior_stdev = 0.01,
    allowed_restaurant_capacity = 0.25, #1.0, # valid values: {1.0, 0.50, 0.25}
    spreading_rate = normal_ci(2.41, 3.90, 20)
#    spreading_rate = normal_ci(0.5, 1.5, 20)
)

sc = 10

population_size = 1000
simulation_cycles = 360 # days
seed = 31415
np.random.seed(seed)

params = copy.deepcopy(common_parameters)


By changing the coverage of school and work districts over the home districts we can make the disease spread faster or slower as desired .  
def setup_grid_layout(model, population_size,
        home_grid_height, home_grid_width,work_height,work_width, school_height, school_width):
    

Makes a grid of homogeneous home districts, overlaid by school and work districts.
    

home_grid_height is the number of home districts high the grid is, and
    

home_grid_width is the nmber of home districts wide the grid is
    

school height and work height are how many home districts high a school
    

district and work are respectively, and the same for their length.
    

each begins in grid 0,0 and cover the orignal home district grid.
    

Persons assigned to the home districts are also assigned to the school
    

and work districts that cover them. The parameters determine the amount
    

of leakage across groups of people.  With parameters (10,10,1,1,1,1), you get 100
    

completely separated districts with no leakage.  With parameters (6,6,2,2,3,3), illustrated below, you
    

get a grid where every one is connected to everyone else, but there is a
    

degree of separation.  For example, a person in home district (0,0) can be infected
    

by a person in (5,5) but it would be bridged by three infections, slowing the
    

virus down.  Larger sizes for work and school districts enable faster spread. Fastest
    

spread occurs with parameters (1,1,1,1,1,1) or equivalently (10,10, 10,10,10,10)
    

or any of the same number
    

Since this is just a way to allocate human interactions, no label is needed and
    

the grid need not be saved, for interactions to occur, although this informamtion
    

may be useful for visualizations. 


 


<img src="grids.jpg">

 Without locality, the defaults to multiple_run are :  home_grid_height = 1, home_grid_width = 1,work_height = 1, work_width =1, school_height=1, school_width=1.  


In [9]:
from platform import python_version

print(python_version())

3.6.8


In [ ]:
import logging
def setLevel(level):
    logger1 = logging.getLogger()
    logger1.setLevel(level)
params.params['temperature'] = -1
params.params['num_communities'] = 12
seeds = [9675]
setLevel(logging.CRITICAL)
multiple_runs(params,population_size,simulation_cycles,num_runs = 30,fname = "duong10000-1", 
              do_print = True, 
              home_grid_height = 5, home_grid_width = 5, school_home_list = 
              [[(0,0),(0,1),(1,1),(2,1),(2,0),(1,0)],
               [(0,2),(0,3),(0,4),(1,3),(2,3),(2,2),(1,2)],
               [(1,4),(2,4),(3,4),(4,4)],
               [(3,3),(4,3),(4,2),(3,2),(3,1),(4,1),(4,0),(3,0)]],
              work_home_list = 
              [[(0,0),(0,1),(1,1),(2,1),(2,0),(1,0),
               (0,2),(0,3),(0,4),(1,3),(2,3),(2,2),(1,2),
               (1,4),(2,4),(3,4),(4,4),
               (3,3),(4,3),(4,2),(3,2),(3,1),(4,1),(4,0),(3,0)]])

Save these seeds if you want to rerun a scenario
[4651, 82, 6776, 1237, 2351, 486, 8491, 4051, 7447, 5669, 4360, 944, 1492, 3116, 4449, 2247, 1902, 6240, 4726, 5805, 9492, 3425, 1129, 1677, 2010, 4727, 2143, 7774, 5796, 8351]
Average similarity between family members is 0.98564204001478 at temperature -1
tup_vec1 not in keepset of size 44
tup_vec1 not in keepset of size 25
tup_vec1 not in keepset of size 6
tup_vec1 not in keepset of size 10
tup_vec1 not in keepset of size 34
tup_vec1 not in keepset of size 15
tup_vec1 not in keepset of size 39
tup_vec1 not in keepset of size 44
tup_vec1 not in keepset of size 43
tup_vec1 not in keepset of size 4
tup_vec1 not in keepset of size 48
tup_vec1 not in keepset of size 9
tup_vec1 not in keepset of size 53
tup_vec1 not in keepset of size 14
tup_vec1 not in keepset of size 18
tup_vec1 not in keepset of size 62
tup_vec1 not in keepset of size 43
tup_vec1 not in keepset of size 64
tup_vec1 not in keepset of size 45
tup_vec1 not in keepset of size 

tup_vec1 not in keepset of size 53
tup_vec1 not in keepset of size 34
tup_vec1 not in keepset of size 57
tup_vec1 not in keepset of size 38
Average similarity between family and home is 0.9919274574296434 at temperature -1
Average similarity between students and their classroom is 0.7238309800450355 at temperature -1
Average similarity between workers is 0.746901665071089 at temperature -1
Average friend similarity for adults: 0.9804736809501465 for kids: 0.9701478992345496
run with seed 4651:
avg restaurant similarity 0.24052708483782373
avg restaurant similarity 0.2745079408174291
avg restaurant similarity 0.2678102929288114
avg restaurant similarity 0.16628650072371323
avg restaurant similarity 0.39870830264155405
avg restaurant similarity 0.17796114718402392
avg restaurant similarity 0.22941686843689651
avg restaurant similarity 0.2385274951856539
avg restaurant similarity 0.21005401495580345
avg restaurant similarity 0.2565877556867451
avg restaurant similarity 0.2860581863598983


avg restaurant similarity 0.18304444865222202
avg restaurant similarity 0.11509456023927002
avg restaurant similarity 0.18403861729559842
avg restaurant similarity 0.15080308339027212
avg restaurant similarity 0.058209810538159594
avg restaurant similarity 0.30062410663208117
avg restaurant similarity 0.15289898860778853
avg restaurant similarity 0.1286527915735875
avg restaurant similarity 0.18745232743177392
avg restaurant similarity 0.18807675965415374
avg restaurant similarity 0.27700650915561725
avg restaurant similarity 0.2911810081489885
avg restaurant similarity 0.26733024107264736
avg restaurant similarity 0.22440470927371262
avg restaurant similarity 0.24408120347772336
avg restaurant similarity 0.2730590261779848
avg restaurant similarity 0.3225834288057557
avg restaurant similarity 0.14922801727598844
avg restaurant similarity 0.142160749673119
avg restaurant similarity 0.18586117973537405
avg restaurant similarity 0.3021820189327243
avg restaurant similarity 0.191123524728

tup_vec1 not in keepset of size 10
tup_vec1 not in keepset of size 53
tup_vec1 not in keepset of size 34
tup_vec1 not in keepset of size 15
tup_vec1 not in keepset of size 58
tup_vec1 not in keepset of size 44
tup_vec1 not in keepset of size 25
tup_vec1 not in keepset of size 6
tup_vec1 not in keepset of size 10
tup_vec1 not in keepset of size 74
tup_vec1 not in keepset of size 35
tup_vec1 not in keepset of size 16
tup_vec1 not in keepset of size 40
tup_vec1 not in keepset of size 21
tup_vec1 not in keepset of size 2
tup_vec1 not in keepset of size 43
tup_vec1 not in keepset of size 24
tup_vec1 not in keepset of size 5
tup_vec1 not in keepset of size 49
tup_vec1 not in keepset of size 10
tup_vec1 not in keepset of size 14
tup_vec1 not in keepset of size 38
tup_vec1 not in keepset of size 19
tup_vec1 not in keepset of size 44
tup_vec1 not in keepset of size 25
tup_vec1 not in keepset of size 6
tup_vec1 not in keepset of size 10
tup_vec1 not in keepset of size 34
tup_vec1 not in keepset 

avg restaurant similarity 0.048728889517254625
avg restaurant similarity 0.06596902775385254
avg restaurant similarity 0.1298360140050255
avg restaurant similarity 0.2963241869451972
avg restaurant similarity 0.040591306744160116
avg restaurant similarity 0.08585556127325351
avg restaurant similarity 0.1941217876993508
avg restaurant similarity 0.18815404401710073
avg restaurant similarity 0.0281377581597912
avg restaurant similarity 0.08212685858209677
avg restaurant similarity 0.20416099299613777
avg restaurant similarity 0.2273028438505291
avg restaurant similarity 0.22930850879240763
avg restaurant similarity 0.38506460404705806
avg restaurant similarity 0.3169567815373671
avg restaurant similarity 0.2902652405094613
avg restaurant similarity 0.17885859289924466
avg restaurant similarity 0.352601486673365
avg restaurant similarity 0.18438179089922424
avg restaurant similarity 0.2684556530676128
avg restaurant similarity 0.2465679725021877
avg restaurant similarity -0.02217397206958

In [ ]:
import logging
def setLevel(level):
    logger1 = logging.getLogger()
    logger1.setLevel(level)
params.params['temperature'] = -0.999
params.params['num_communities'] = 12
seeds = [9675]
setLevel(logging.CRITICAL)
multiple_runs(params,population_size,simulation_cycles,num_runs = 30,fname = "duong10000-1", 
              do_print = True, 
              home_grid_height = 5, home_grid_width = 5, school_home_list = 
              [[(0,0),(0,1),(1,1),(2,1),(2,0),(1,0)],
               [(0,2),(0,3),(0,4),(1,3),(2,3),(2,2),(1,2)],
               [(1,4),(2,4),(3,4),(4,4)],
               [(3,3),(4,3),(4,2),(3,2),(3,1),(4,1),(4,0),(3,0)]],
              work_home_list = 
              [[(0,0),(0,1),(1,1),(2,1),(2,0),(1,0),
               (0,2),(0,3),(0,4),(1,3),(2,3),(2,2),(1,2),
               (1,4),(2,4),(3,4),(4,4),
               (3,3),(4,3),(4,2),(3,2),(3,1),(4,1),(4,0),(3,0)]])

In [ ]:
import logging
def setLevel(level):
    logger1 = logging.getLogger()
    logger1.setLevel(level)
params.params['temperature'] = 1
params.params['num_communities'] = 12
seeds = [9675]
setLevel(logging.CRITICAL)
multiple_runs(params,population_size,simulation_cycles,num_runs = 30,fname = "duong10000-1", 
              do_print = True, 
              home_grid_height = 5, home_grid_width = 5, school_home_list = 
              [[(0,0),(0,1),(1,1),(2,1),(2,0),(1,0)],
               [(0,2),(0,3),(0,4),(1,3),(2,3),(2,2),(1,2)],
               [(1,4),(2,4),(3,4),(4,4)],
               [(3,3),(4,3),(4,2),(3,2),(3,1),(4,1),(4,0),(3,0)]],work_home_list = 
              [[(0,0),(0,1),(1,1),(2,1),(2,0),(1,0),
               (0,2),(0,3),(0,4),(1,3),(2,3),(2,2),(1,2),
               (1,4),(2,4),(3,4),(4,4),
               (3,3),(4,3),(4,2),(3,2),(3,1),(4,1),(4,0),(3,0)]])

In [ ]:
import logging
def setLevel(level):
    logger1 = logging.getLogger()
    logger1.setLevel(level)
params.params['temperature'] = 0
params.params['num_communities'] = 12
seeds = [9675]
setLevel(logging.CRITICAL)
multiple_runs(params,population_size,simulation_cycles,num_runs = 30,fname = "duong10000-1", 
              do_print = True, 
              home_grid_height = 5, home_grid_width = 5, school_home_list = 
              [[(0,0),(0,1),(1,1),(2,1),(2,0),(1,0)],
               [(0,2),(0,3),(0,4),(1,3),(2,3),(2,2),(1,2)],
               [(1,4),(2,4),(3,4),(4,4)],
               [(3,3),(4,3),(4,2),(3,2),(3,1),(4,1),(4,0),(3,0)]],work_home_list = 
              [[(0,0),(0,1),(1,1),(2,1),(2,0),(1,0),
               (0,2),(0,3),(0,4),(1,3),(2,3),(2,2),(1,2),
               (1,4),(2,4),(3,4),(4,4),
               (3,3),(4,3),(4,2),(3,2),(3,1),(4,1),(4,0),(3,0)]])

In [ ]:
import logging
def setLevel(level):
    logger1 = logging.getLogger()
    logger1.setLevel(level)
params.params['temperature'] = -0.9
params.params['num_communities'] = 12
seeds = [9675]
setLevel(logging.CRITICAL)
multiple_runs(params,population_size,simulation_cycles,num_runs = 30,fname = "duong10000-1", 
              do_print = True, 
              home_grid_height = 5, home_grid_width = 5, school_home_list = 
              [[(0,0),(0,1),(1,1),(2,1),(2,0),(1,0)],
               [(0,2),(0,3),(0,4),(1,3),(2,3),(2,2),(1,2)],
               [(1,4),(2,4),(3,4),(4,4)],
               [(3,3),(4,3),(4,2),(3,2),(3,1),(4,1),(4,0),(3,0)]],work_home_list = 
              [[(0,0),(0,1),(1,1),(2,1),(2,0),(1,0),
               (0,2),(0,3),(0,4),(1,3),(2,3),(2,2),(1,2),
               (1,4),(2,4),(3,4),(4,4),
               (3,3),(4,3),(4,2),(3,2),(3,1),(4,1),(4,0),(3,0)]])